In [ ]:
import pandas as pd
import numpy as np
import ta.trend
from Utils import extract_symbol_timeframe

# === Параметры стратегии ===
fast = 9
slow = 26
signal = 9
tp = 0.5  # take-profit %
stop_loss_pct = 99  # условный высокий стоп

# === Загрузка данных ===

path = " " # Format filename = {symbol}USDT_timeframe_data.csv, example: "BTCUSDT_1h_from_1_Jan_2020.csv"
df = pd.read_csv(path)
symbol, timeframe = extract_symbol_timeframe(path)
indicator_name = "MACD_with_logic_below_0"


df["open_time"] = pd.to_datetime(df["open_time"], utc=True)
df.dropna(inplace=True)

# === Расчёт MACD ===
df["macd"] = ta.trend.macd(df["close"], window_slow=slow, window_fast=fast)
df["macd_signal"] = ta.trend.macd_signal(df["close"], window_slow=slow, window_fast=fast, window_sign=signal)
df.dropna(inplace=True)

# === Массивы ===
macd = df["macd"].to_numpy()
macd_signal = df["macd_signal"].to_numpy()
closes = df["close"].to_numpy(dtype=np.float64)
highs = df["high"].to_numpy(dtype=np.float64)
lows = df["low"].to_numpy(dtype=np.float64)
opens = df["open"].to_numpy(dtype=np.float64)
times = df["open_time"].to_numpy(dtype="datetime64[ns]")

# === Сигналы входа: пересечение MACD снизу вверх и MACD ниже 0 ===
cross_up = (macd[:-1] < macd_signal[:-1]) & (macd[1:] > macd_signal[1:]) & (macd[1:] < 0)
entry_idx = np.where(cross_up)[0] + 1

# === Обработка сделок ===
results = []
last_exit = 0

for i in entry_idx:
    if i <= last_exit:
        continue

    entry_price = closes[i]
    take_profit = entry_price * (1 + tp / 100)
    stop_loss = entry_price * (1 - stop_loss_pct / 100)
    entry_time = times[i]

    for j in range(i + 1, len(closes)):
        if highs[j] >= take_profit:
            exit_price = take_profit
            exit_time = times[j]
            exit_type = "take_profit"
            break
        elif lows[j] <= stop_loss:
            exit_price = stop_loss
            exit_time = times[j]
            exit_type = "stop_loss"
            break
    else:
        break

    duration = (exit_time - entry_time).astype("timedelta64[m]").astype(int)
    profit_pct = (exit_price / entry_price - 1) * 100

    results.append({
        "entry_time": times[i],
        "open": opens[i],
        "close": closes[i],
        "high": highs[i],
        "low": lows[i],
        "macd": macd[i],
        "macd_signal": macd_signal[i],
        "exit_time": exit_time,
        "duration_min": duration,
        "exit_type": exit_type,
        "profit_pct": round(profit_pct, 2)
    })

    last_exit = j

# === Результирующий датафрейм ===
deals_df = pd.DataFrame(results)
print("✅ Сделок найдено:", len(deals_df))
print(f"Криптовалюта: {symbol}\nТаймфрейм: {timeframe}\nСтратегия: {indicator_name}")
deals_df.tail(10)
